In [133]:
import os
import sys
import re



In [227]:
# Create some data structures and variables for our program
budget = {}
profit_loss_total = 0
profit_loss_high = []
profit_loss_low = []

#Opens the file "budget_data.csv"
with open("budget_data.csv", mode = 'r') as f:

#Creates an iterable "i" that we will use for error references
    i = 0
    
    
#Starts the main loop
    for line in f:
        i += 1 
        if i > 0:
             
            #Checks for a specific concantenation of 3 letters for month, 4 digits for year, a comma,
            #Then everything after that in the line (which should only be numbers). 
            budget_item_match = re.search(r'(\w\w\w-\d\d\d\d),(.+)', line)

            #This is simple enough but creates one issue. in the second group (.+), we just grab everything after the comma
            #Some numbers have a negative sign, some do not. We need to make certain that our second item is a number (imagine someone typed in "n/a" 
            # or accidently entered a "Q" instead of "1")
            if budget_item_match:
                #Set a variable equal to our P/l as a string object, so that we can iterate through it real quick and make sure it fits the right format
                check_pl_data = budget_item_match.group(2)
                #Here we loop through each 'ch' character in the string check_data. If the ch is a negative sign or a digit, we pass. 
                for ch in check_pl_data:
                    if ch == "-":
                        pass
                    elif ch.isdigit():
                        pass
                    else:
                        print(f"Error on line # {i}: {line} -- {ch} is causing an error")
                        print(f"File must include 1 header row in first line, date in first column (MON-YEAR, ex: JAN-2001), and profit/loss in 2nd colummn without $ (ex: -189000)")
                        print("Please remove or reformat line, save the file, and retry")
                        sys.exit(1)
                #Now we want to start tracking some P/L data as it's coming in:
                if profit_loss_total == 0:
                    profit_loss_total = int(budget_item_match.group(2))
                    profit_loss_high = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    profit_loss_low = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    new_key = 1
                    budget[new_key] = (budget_item_match.group(1),int(budget_item_match.group(2)))
                else:
                    #At this point, I want to force our budget{} keys to be sequential regardless of how many blank lines there are:
                    new_key += 1 
                    budget[new_key] = (budget_item_match.group(1),int(budget_item_match.group(2)))
                    profit_loss_total = profit_loss_total + int(budget_item_match.group(2))
                    #Tracks the most recent high mark for monthly p/l
                    if profit_loss_high[-1] <= int(budget_item_match.group(2)):
                        profit_loss_high = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
                    #Tracks the most recent low mark for monthly p/l
                    elif profit_loss_low[-1] >= int(budget_item_match.group(2)):
                        profit_loss_low = [budget_item_match.group(1) ,int(budget_item_match.group(2))]
            # First we check for a line we don't know what to do with:    
            if not budget_item_match:
                #Next we skip the first row, where the header is expected anyway and we don't need those
                if i == 1:
                    pass
                # Next we check for an empty line, or an empty line with whitespaces (which is why using line == '\n' would not comprehensively solve the issue)
                elif len(line.strip()) == 0:
                    print(f"Line # {i} is empty")
                #Finally if there is still an issue, we error out and print the line with an issue, formatting instructions, and directions to user
                else:
                    print(f"Error on line # {i}: {line}")
                    print(f"File must include 1 header row in FIRST LINE, date in first column (MON-YEAR, ex: JAN-2001), and profit/loss in 2nd colummn without $ (ex: -189000)")
                    print("Please remove or reformat line, save the file, and retry")
                    sys.exit(1)

#Now we need to calculate the proper average. A dictionary isn't sorted inherently, but since we iterated new_key, all the key-value pairs match up with the 
#monthly progression of the data. So step 1) grab all the values and turn them into a list:
budget_items_object = budget.values()
budget_tuples_list = list(budget_items_object)
p_l_list = []
deltas = []
#step 2) loop through all the tuples and grab the profit from each month, creating a list of all the SEQUENTIAL CHANGES:
for v in range(len(budget_tuples_list)):
    budget_tup = budget_tuples_list[v]
    p_l_list.append(budget_tup[1])
#step 3) build a list of all the month-to-month deltas in p/l:
    if v >= 1:
        deltas.append(p_l_list[v] - p_l_list[v-1])
average_deltas = (sum(deltas)/len(deltas))


print(new_key)
print(profit_loss_total)
print(profit_loss_high)
print(profit_loss_low)

print(average_deltas)

86
38382578
['Feb-2012', 1170593]
['Sep-2013', -1196225]
-2315.1176470588234


In [93]:

# budget = {}
# pl_list = []
# pl_change = []

# with open("budget_data.csv", newline=None) as f:

#     i = -1

#     for line in f: 
#         i += 1 #this will let us skip the header
#         if i >= 0:
#             _match = re.search(r'(\w\w\w-\d\d\d\d),(.+)', line)
#             if _match:
#                 budget[_match.group(1)] = int(_match.group(2))
#     profit_loss_total = 0
#     profit_loss_change = 0
#     profit_loss_high = 0
#     profit_loss_low = 0
#     for k in budget:
#         if budget[k] > profit_loss_high:
#             profit_loss_high = budget[k]
#             profit_loss_high_date = k
#         elif budget[k] < profit_loss_low:
#             profit_loss_low = budget[k]
#             profit_loss_low_date = k
#         pl_list.append(budget[k])
#         profit_loss_total = profit_loss_total + budget[k]       
#         if len(pl_list)>1:
#             profit_loss_change = profit_loss_change + (pl_list[-1] - pl_list[-2])
#             pl_change.append(pl_list[-1] - pl_list[-2])



    # print('Financial Analysis')
    # print('----------------------------')
    # print('Total Months:' + ' ' + str(i))
    # print('Total:' + ' ' + '$' + str(profit_loss_total) )
    # print('Average Change:'+ ' ' + '$'+ str(round(profit_loss_change/len(pl_list),2)))
    # print('Greatest Increase in Profits: '+ profit_loss_high_date + " "+"("+"$"+str(profit_loss_high)+")" )
    # print('Greatest Decrease in Profits: '+ profit_loss_low_date + " "+"("+"$"+str(profit_loss_low)+")" )
    
 
 


{'Jan-2010': 867884, 'Feb-2010': 984655, 'Mar-2010': 322013, 'Apr-2010': -69417, 'May-2010': 310503, 'Jun-2010': 522857, 'Jul-2010': 1033096, 'Aug-2010': 604885, 'Sep-2010': -216386, 'Oct-2010': 477532, 'Nov-2010': 893810, 'Dec-2010': -80353, 'Jan-2011': 779806, 'Feb-2011': -335203, 'Mar-2011': 697845, 'Apr-2011': 793163, 'May-2011': 485070, 'Jun-2011': 584122, 'Jul-2011': 62729, 'Aug-2011': 668179, 'Sep-2011': 899906, 'Oct-2011': 834719, 'Nov-2011': 132003, 'Dec-2011': 309978, 'Jan-2012': -755566, 'Feb-2012': 1170593, 'Mar-2012': 252788, 'Apr-2012': 1151518, 'May-2012': 817256, 'Jun-2012': 570757, 'Jul-2012': 506702, 'Aug-2012': -1022534, 'Sep-2012': 475062, 'Oct-2012': 779976, 'Nov-2012': 144175, 'Dec-2012': 542494, 'Jan-2013': 359333, 'Feb-2013': 321469, 'Mar-2013': 67780, 'Apr-2013': 471435, 'May-2013': 565603, 'Jun-2013': 872480, 'Jul-2013': 789480, 'Aug-2013': 999942, 'Sep-2013': -1196225, 'Oct-2013': 268997, 'Nov-2013': -687986, 'Dec-2013': 1150461, 'Jan-2014': 682458, 'Feb-2014